In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


# Using linear model in Keras

In [2]:
x = random((30, 2))
y = np.dot(x, [3.0, 4.0]) + 5.0

In [3]:
x[:5],y[:5]

(array([[ 0.7933,  0.2364],
        [ 0.1868,  0.197 ],
        [ 0.7556,  0.8337],
        [ 0.6765,  0.1819],
        [ 0.7416,  0.1918]]),
 array([  8.3256,   6.3485,  10.6016,   7.7569,   7.992 ]))

In [4]:
# create model
lm = Sequential([Dense(1, input_shape=(2,))])

In [5]:
lm.compile(optimizer=SGD(lr=0.01), loss='mse')

In [6]:
lm.evaluate(x, y, verbose=0)

75.819633483886719

In [32]:
lm.fit(x, y, nb_epoch=10, batch_size = 1)

Epoch 1/10
30/30 [==============================] - 0s - loss: 7.0062e-04     
Epoch 2/10
30/30 [==============================] - 0s - loss: 6.6033e-04 
Epoch 3/10
30/30 [==============================] - 0s - loss: 6.1846e-04     
Epoch 4/10
30/30 [==============================] - 0s - loss: 5.7997e-04     
Epoch 5/10
30/30 [==============================] - 0s - loss: 5.4612e-04     
Epoch 6/10
30/30 [==============================] - 0s - loss: 5.0342e-04     
Epoch 7/10
30/30 [==============================] - 0s - loss: 4.7989e-04     
Epoch 8/10
30/30 [==============================] - 0s - loss: 4.5131e-04     
Epoch 9/10
30/30 [==============================] - 0s - loss: 4.2237e-04 
Epoch 10/10
30/30 [==============================] - 0s - loss: 3.9513e-04     


In [33]:
lm.evaluate(x, y, verbose=0)

0.00037243173574097455

In [34]:
lm.get_weights()

[array([[ 2.9453],
        [ 3.9595]], dtype=float32), array([ 5.0496], dtype=float32)]

In [22]:
# change the learning rate after one round fit
lm.optimizer.lr = 0.01

# Training model on predictions from CNN

The overall approach is:

1. get the true labels for our images
2. get the 1000 categories for our images from VGG16
3. feed the 1000 categories as input to our linear model

In [35]:
path = 'data/dogs-vs-cats-redux-kernels-edition/'

In [36]:
from vgg16 import Vgg16

In [37]:
vgg = Vgg16()

In [38]:
model = vgg.model

In [45]:
train_batches = get_batches(path + 'train')
valid_batches = get_batches(path + 'valid')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [44]:
train_data = get_data(path + 'train')
valid_data = get_data(path + 'valid')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [46]:
train_data.shape, valid_data.shape

((23000, 3, 224, 224), (2000, 3, 224, 224))

In [47]:
save_array(path + 'model/train_data.bc', train_data)
save_array(path + 'model/valid_data.bc', valid_data)

In [48]:
train_data = load_array(path + 'model/train_data.bc')
valid_data = load_array(path + 'model/valid_data.bc')

In [49]:
train_data.shape, valid_data.shape

((23000, 3, 224, 224), (2000, 3, 224, 224))

In [53]:
train_batches.classes.shape

(23000,)

In [54]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1, 1)).todense())

In [66]:
test1 = np.array([1, 3, 2, 3, 4])
onehot(test1)

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [67]:
val_classes = valid_batches.classes
trn_classes = train_batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [68]:
trn_labels.shape, val_labels.shape

((23000, 2), (2000, 2))

In [69]:
trn_labels[:4]

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [70]:
batch_size = 64

In [71]:
trn_features = model.predict(train_data, batch_size = batch_size)
val_features = model.predict(valid_data, batch_size = batch_size)

In [72]:
trn_features.shape, val_features.shape

((23000, 1000), (2000, 1000))

In [73]:
save_array(path + 'model/train_lastlayer_features.bc', trn_features)
save_array(path + 'model/valid_lastlayer_features.bc', val_features)

In [74]:
trn_features = load_array(path + 'model/train_lastlayer_features.bc')
val_features = load_array(path + 'model/valid_lastlayer_features.bc')

In [75]:
lm = Sequential([Dense(2, input_shape=(1000,), activation='softmax')])
lm.compile(optimizer=RMSprop(lr = 0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [76]:
lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/3
23000/23000 [==============================] - 0s - loss: 0.1006 - acc: 0.9659 - val_loss: 0.0743 - val_acc: 0.9745
Epoch 2/3
23000/23000 [==============================] - 0s - loss: 0.0811 - acc: 0.9746 - val_loss: 0.0848 - val_acc: 0.9740
Epoch 3/3
23000/23000 [==============================] - 0s - loss: 0.0830 - acc: 0.9760 - val_loss: 0.0929 - val_acc: 0.9760


In [77]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_5 (Dense)                  (None, 2)             2002        dense_input_2[0][0]              
Total params: 2,002
Trainable params: 2,002
Non-trainable params: 0
____________________________________________________________________________________________________
